# HANDL 

The goal of HANDL is to support the transfer of biological knowledge across species.  It does so by constructing explicit measures of functional similarity between proteins in different species.   HANDL (Homology Assessment across Networks using Diffusion and Landmarks) creates a functional embedding in which proteins from different species are embedded in the same vector space. In the paper [1] it is shown that inner products in this space _(HANDL homology scores)_, as well as the vectors themselves _(HANDL embeddings)_, capture functional similarity across species and are useful for a variety of functional tasks.

[1] [A Multi-Species Functional Embedding Integrating Sequence and
Network Structure](https://www.biorxiv.org/content/early/2017/12/04/229211), Mark D.M. Leiserson, Jason Fan, Anthony Cannistra, Inbar Fried, Tim Lim, Thomas Schaffner, Mark Crovella, and Benjamin Hescott, _Proceedings of RECOMB 2018,_ Paris, France.

## Background

An overview of HANDL is shown in the figure below.  Starting with a source and a target network, HANDL computes diffusion kernels for each network. The diffusion kernels are factored, resulting in a representation in which each node becomes a vector, and inner products correlate with functional similarities. HANDL then solves a linear system to create a single embedding of both vector sets, allowing for functional comparisons between proteins across the two networks.

![HANDL Overview](figures/handl-methods.jpg)

As shown in the figure, HANDL works with the notion of a _source_ species and one or more _target_ species.  The proteins of the target species are embedded into a space defined by the source species.

## This workbook

This workbook illustrates how to compute HANDL embeddings and HANDL homology scores as described in [1].  As a demonsration, we compute functional similarity between proteins in _S. cerevisiae_ (Sc) and _S. pombe_ (Sp).

## Configuration 

This workbook assumes that the [HANDL demo repository](https://github.com/theJasonFan/HANDL) has been downloaded, and that data has been retrieved and prepared as described in the __Data__ section of the [README](https://github.com/theJasonFan/HANDL/blob/master/README.md).

In [1]:
import numpy as np
import networkx as nx
import scipy
import sklearn
import sklearn.neighbors
import matplotlib.pyplot as plt

Some of the functions we will use are in associated libraries.

In [2]:
import util
from network import regularized_laplacian, rkhs_factor

Configuration parameters:
* `source`: Source species
* `target`: Target species
* `lam`: Value of $\lambda$ used in computing the Regularized Laplacian
* `data_dir`: Directory holding all data
* `n_landmarks`: Number of landmarks to use in constructing the embedding 

In [3]:
source = 'sc'
target = 'sp'
lam = 0.05
data_dir = '../data'
n_landmarks = 400

## HANDL embedding

The first stage of the HANDL embedding involves the following steps:

1. Construct PPI networks for source and target species
2. Reduce each network to its 2-core (see paper for explanation).
3. Compute the Laplacian $L$ and Regularized Laplacian $D = (I + \lambda L)^{-1}$ for each species.
4. Factor the Regularized Laplacian to get HANDL embedding vectors $D = CC^T$.

In [4]:
nodes = {}
D = {}
C = {}
for species in [source, target]:
    ppi_edgelist = '{}/ppi/biogrid/{}/biogrid-{}-std-network.txt'.format(
        data_dir, species, species)
    G = nx.read_edgelist(ppi_edgelist, encoding='ascii')
    G = util.simple_two_core(G)
    nodes[species] = sorted(G.nodes())
    L = np.array(nx.laplacian_matrix(G[species], nodelist=nodes[species]).todense())
    D[species] = regularized_laplacian(L, lam)
    C[species] = rkhs_factor(D[species])

2017-12-23 14:30:57,180 util.py         INFO      : PPI info - # Nodes: 5893, # Edges: 492146
2017-12-23 14:30:57,181 util.py         INFO      : Computing 2 core
2017-12-23 14:31:00,275 util.py         INFO      : 2 core info - # Nodes: 5865, # Edges: 492118
2017-12-23 14:31:00,276 util.py         INFO      : 2 core removed 28 nodes and 28 edges


KeyError: 'sc'

The second stage of the HANDL embedding uses known homologs between the two species.  First we read in the set of all known homologs.

In [ ]:
# assume the order in homolog file is source node, target node
homolog_list = '{}/homologs/sc-sp/sc-sp-homologs.txt'.format(
    data_dir)
source_homs = []
target_homs = []
with open(homolog_list, 'r') as hom_f:
    for line in hom_f:
        source_hom, target_hom = line.split()
        if (source_hom in nodes[source]) and (target_hom in nodes[target]):
            source_homs.append(source_hom)
            target_homs.append(target_hom)

In [ ]:
source_homolog_indices = [nodes[source].index(node) for node in source_homs]
target_homolog_indices = [nodes[target].index(node) for node in target_homs]
source_landmarks = source_homs[:n_landmarks]
target_landmarks = target_homs[:n_landmarks]

source_landmark_indices = source_homolog_indices[:n_landmarks]
source_non_landmark_indices = source_homolog_indices[n_landmarks:]
target_landmark_indices = target_homolog_indices[:n_landmarks]
target_non_landmark_indices = target_homolog_indices[n_landmarks:]

Embedding of target species

select corresponding rows in each vector set and regress

In [ ]:
target_handl_C = np.linalg.pinv(C[source][source_landmark_indices,:]).dot(
    D[target][target_landmark_indices,:])

## HANDL homology scores

In [ ]:
D_cross_species = C[source].dot(target_handl_C)

## Plot homolog density (Figure 2 from paper)

Parameters for plot

In [ ]:
plot_lim = 0.8
font_size = 12
line_width = 2

For this plot we want to work with dissimilarities

In [ ]:
Dissim = 1./D_cross_species

Normalize

In [ ]:
Dissim = Dissim / np.mean(Dissim)

We don't want to use known landmark-homologs so we remove those entries from the HANDL score matrix

In [ ]:
source_valid = [r for r in range(Dissim.shape[0])
                        if r not in source_landmark_indices]
target_valid = [r for r in range(Dissim.shape[1])
                        if r not in target_landmark_indices]
Dissim = Dissim[source_valid][:,target_valid]

In [ ]:
# update the names of rows and columns
source_nodes = [n for i, n in enumerate(nodes[source]) if i in source_valid]
target_nodes = [n for i, n in enumerate(nodes[target]) if i in target_valid]

# update these indices as well
source_non_landmark_indices = [source_nodes.index(node) for node in source_homs
                                       if node not in source_landmarks]
target_non_landmark_indices = [target_nodes.index(node) for node in target_homs
                                       if node not in target_landmarks]
    

In [ ]:
# get HANDL scores for Non-landmark homologs
NLH = Dissim [source_non_landmark_indices,target_non_landmark_indices]

In [ ]:
# create kernel density estimator
kde = sklearn.neighbors.KernelDensity(kernel='gaussian',
                                              bandwidth = plot_lim / 20.)

In [ ]:
# need to add another dimension as required by sklearn
# arrays passed to kde must be 2-dimensional
X_plot = np.reshape(np.linspace(0, plot_lim, 500), (-1, 1))

In [ ]:
# Density of non-landmark homolog pairs
kde.fit(np.reshape(np.ravel(NLH), (-1, 1)))
NLHdens = kde.score_samples(X_plot)

In [ ]:
# Density of all pairs
Dsamp = np.random.choice(np.ravel(Dissim),100000)
kde.fit(np.reshape(Dsamp, (-1, 1)))
Ddens = kde.score_samples(X_plot)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(X_plot[:,0], np.exp(NLHdens), lw = line_width,
                 label = 'Homolog Pairs')
plt.plot(X_plot[:,0], np.exp(Ddens), '-.',
                 lw = line_width, ls = (0,(5,2)), label = 'All Pairs')
plt.ylabel('Density', size = font_size)
plt.xlabel('HANDL dissimilarity', size = font_size)
plt.legend(loc='best', fontsize = font_size)